In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2012_to_2014.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2009_to_2011.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2005_to_2007.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accident_coords_update.ipynb
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/ukTrafficAADF.csv
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/Areas.shp
/kaggle/input/2000-16-traffic-flow-england-scotland-wales/Local_Authority_Districts_Dec_2016.geojson


In [2]:
dataset1=pd.read_csv('/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2005_to_2007.csv')
#dataset2=pd.read_csv('/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2009_to_2011.csv')
#dataset3=pd.read_csv('/kaggle/input/2000-16-traffic-flow-england-scotland-wales/accidents_2012_to_2014.csv')

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (31) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
dataset1.info()
#dataset2.info()
#dataset3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570011 entries, 0 to 570010
Data columns (total 33 columns):
 #   Column                                       Non-Null Count   Dtype  
---  ------                                       --------------   -----  
 0   Accident_Index                               570011 non-null  object 
 1   Location_Easting_OSGR                        569910 non-null  float64
 2   Location_Northing_OSGR                       569910 non-null  float64
 3   Longitude                                    569910 non-null  float64
 4   Latitude                                     569910 non-null  float64
 5   Police_Force                                 570011 non-null  int64  
 6   Accident_Severity                            570011 non-null  int64  
 7   Number_of_Vehicles                           570011 non-null  int64  
 8   Number_of_Casualties                         570011 non-null  int64  
 9   Date                                         570011 non-nul

In [4]:
df1=dataset1.drop(['Accident_Index',
                   'Local_Authority_(District)',
                   'Local_Authority_(Highway)',
                   '1st_Road_Number',
                   'Junction_Detail',
                   '2nd_Road_Number',
                   'Year',
                   'Location_Easting_OSGR',
                   'Location_Northing_OSGR',
                   'Longitude',
                   'Latitude',
                   'LSOA_of_Accident_Location'
                  ],axis=1)

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 570011 entries, 0 to 570010
Data columns (total 21 columns):
 #   Column                                       Non-Null Count   Dtype 
---  ------                                       --------------   ----- 
 0   Police_Force                                 570011 non-null  int64 
 1   Accident_Severity                            570011 non-null  int64 
 2   Number_of_Vehicles                           570011 non-null  int64 
 3   Number_of_Casualties                         570011 non-null  int64 
 4   Date                                         570011 non-null  object
 5   Day_of_Week                                  570011 non-null  int64 
 6   Time                                         569944 non-null  object
 7   1st_Road_Class                               570011 non-null  int64 
 8   Road_Type                                    570011 non-null  object
 9   Speed_limit                                  570011 non-null  int64 
 

In [10]:
df1['Junction_Control'].isnull().sum()

236945

In [13]:
verif=df1.apply(lambda x:True if x['2nd_Road_Class'] == -1 and x['Junction_Control'] == np.nan else False, axis=1)
verif.value_counts()

False    570011
dtype: int64

In [37]:
#True si pas de 2nd_Road
verif1=df1.apply(lambda x:True if x['2nd_Road_Class'] == -1 else False, axis=1)
verif1.value_counts()

False    328335
True     241676
dtype: int64

In [15]:
df1['2nd_Road_Class'].value_counts()

-1    241676
 6    223284
 3     54931
 5     24342
 4     21601
 1      3739
 2       438
Name: 2nd_Road_Class, dtype: int64

In [16]:
df1['Junction_Control'].value_counts()

Giveway or uncontrolled     271570
Automatic traffic signal     56121
Stop Sign                     4320
Authorised person             1055
Name: Junction_Control, dtype: int64

In [18]:
570011-df1['Junction_Control'].value_counts().sum()

236945

In [32]:
#True si pas d'intersection
verif2=df1.apply(lambda x:True if x['Junction_Control']!='Authorised person' and x['Junction_Control']!='Giveway or uncontrolled' and x['Junction_Control']!='Automatic traffic signal' and x['Junction_Control']!='Stop Sign' else False, axis=1)
verif2.value_counts()

False    333066
True     236945
dtype: int64

In [33]:
verif3=df1.apply(lambda x:True if x['2nd_Road_Class'] == -1 and x['Junction_Control']!='Authorised person' and x['Junction_Control']!='Giveway or uncontrolled' and x['Junction_Control']!='Automatic traffic signal' and x['Junction_Control']!='Stop Sign' else False, axis=1)
verif3.value_counts()

False    333624
True     236387
dtype: int64

In [ ]:
# en faisant la soustraction des 'True' de verif2 et verif3, on obtient le nbr de mesure où 
# il y a 2nd_Road_Class != -1 (donc il y a une route secondaire) mais 
# il n'y a pas de descriptif du control de l'intersection.
#
# De plus, le résultat de cette soustraction est positif donc
# toutes les lignes avec 2nd_Road_Class == -1 correspondent bien à Junction_Control == Null

In [36]:
#True si il y a une 2nd_Road mais pas de description du control de l'intersection
verif4=df1.apply(lambda x:True if x['2nd_Road_Class'] != -1 and x['Junction_Control']!='Authorised person' and x['Junction_Control']!='Giveway or uncontrolled' and x['Junction_Control']!='Automatic traffic signal' and x['Junction_Control']!='Stop Sign' else False, axis=1)
verif4.value_counts()

False    569453
True        558
dtype: int64

In [ ]:
'''
TODO : Drop les 558 lignes correspondantes à l'absence de renseignement sur l'intersection quand il y a 1 deuxième route.
       Junction_Control devient alors une donnée d'intérêt et les valeurs Null deviennent 'no_junction'
'''

In [ ]:
#Objectif : mis en place d'un model prédictif sur la sévérité d'un accident

# donnée de sortie (Y) : Accident_Severity

# données sans intérêt : Accident_Index , Local_Authority_(District) , Local_Authority_(Highway) , 1st_Road_Number , Junction_Detail ,
#                        2nd_Road_Number , Year , Location_Easting_OSGR , Location_Northing_OSGR , Longitude , Latitude , 
#                        LSOA_of_Accident_Location

# données d'intérêt (X) : Police_Force , Number_of_Vehicles , Number_of_Casualties , Date , Day_of_Week , Time ,
#                         1st_Road_Class , Road_Type , Speed_limit , Pedestrian_Crossing-Human_Control , Pedestrian_Crossing-Physical_Facilities ,
#                         Light_Conditions , Weather_Conditions , Road_Surface_Conditions , Special_Conditions_at_Site , Carriageway_Hazards ,
#                         Urban_or_Rural_Area , Did_Police_Officer_Attend_Scene_of_Accident , Junction_Control

'''
TODO :  _ count() les lignes dont 1 attribut est renseigné 'unknown'
        - gestion des valeurs Null
'''